# Executive Briefing

# Reproducible Analysis

## Loading the data

In [7]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import re
import os

from matplotlib.colors import ListedColormap

import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from esda.adbscan import ADBSCAN

import random
random.seed(42)    # For reproducibility
np.random.seed(42) # For reproducibility

# Make numeric display a bit neater
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

#visualisation set
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.mpl.rc("figure", figsize=(10,6))

In [9]:
# Set download URL
url = 'http://data.insideairbnb.com/united-kingdom/england/london/2019-05-05/data/listings.csv.gz'

airbnb = pd.read_csv(url, compression='gzip', low_memory=False, usecols = ['minimum_minimum_nights','minimum_nights','property_type','room_type','longitude','latitude','price','bedrooms','beds','accommodates','neighbourhood','neighbourhood_cleansed'])
print(f"Data frame is {airbnb.shape[0]:,} x {airbnb.shape[1]}")

airbnb.info()

Data frame is 80,767 x 12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80767 entries, 0 to 80766
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighbourhood           80766 non-null  object 
 1   neighbourhood_cleansed  80767 non-null  object 
 2   latitude                80767 non-null  float64
 3   longitude               80767 non-null  float64
 4   property_type           80767 non-null  object 
 5   room_type               80767 non-null  object 
 6   accommodates            80767 non-null  int64  
 7   bedrooms                80730 non-null  float64
 8   beds                    80633 non-null  float64
 9   price                   80767 non-null  object 
 10  minimum_nights          80767 non-null  int64  
 11  minimum_minimum_nights  80767 non-null  int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 7.4+ MB


In [ ]:
# Set download URL
#url = 'https://github.com/YHuj20/i2pBriefing/'

#import pandas as pd
#airbnb = pd.read_csv(url, compression='gzip', low_memory=False)

#print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

emptyhomes = pd.read_csv("Data/All-longterm-vacant-dwellings.csv", low_memory = False)
emptyhomes = emptyhomes[emptyhomes['2018'].notna()]
emptyhomes.info()

In [ ]:
# Set download URL
#url = 'https://github.com/YHuj20/i2pBriefing/'

#import pandas as pd
#airbnb = pd.read_csv(url, compression='gzip', low_memory=False)

#print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

totalhomes = pd.read_csv("Data/Total-dwellings-per-borough.csv", low_memory = False)
totalhomes.info()

In [ ]:
#

In [6]:
for col in airbnb.columns: 
    print(col) 

id
listing_url
scrape_id
last_scraped
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
access
interaction
house_rules
thumbnail_url
medium_url
picture_url
xl_picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_ni

In [ ]:
airbnb.groupby(['neighbourhood','room_type']).count()

In [ ]:
typecount = airbnb.pivot_table('id', 'neighbourhood', 'room_type', aggfunc='count', fill_value=0)
typecount = typecount.rename(columns={"Entire home/apt" : "Entire_Home" , "Private room": "Private_Room", "Shared room": "Shared_Room"})

In [ ]:
typecount['Percent_Entire_Home'] = typecount.Entire_Home/typecount.Entire_Home.sum()
typecount['Percent_Private_Room'] = typecount.Private_Room/typecount.Private_Room.sum()
typecount['Percent_Shared_Room'] = typecount.Shared_Room/typecount.Shared_Room.sum()
typecount['LAD_Name'] = typecount.index
typecount.sample(12, random_state = 42)

In [ ]:
#Join datasets by LA name
#calculate density of entire homes against total homes per borough

typecount=typecount.set_index('LAD_Name').join(totalhomes.set_index('Local Authority'))
typecount['Density_of_Entire_Homes'] = typecount['Entire_Home']/typecount['2018']
typecount['LAD_Name'] = typecount.index
typecount.head()

In [ ]:
#Entire Homes Locations
entirehomeslatlong = airbnb[['latitude', 'longitude', 'room_type']].copy()
entirehomeslatlong.info()

## Mapping Entire AirBnB Homes

In [ ]:
import os
from requests import get
from urllib.parse import urlparse

def cache_data(src:str, dest:str) -> str:
    """Downloads and caches a remote file locally.
    
    The function sits between the 'read' step of a pandas or geopandas
    data frame and downloading the file from a remote location. The idea
    is that it will save it locally so that you don't need to remember to
    do so yourself. Subsequent re-reads of the file will return instantly
    rather than downloading the entire file for a second or n-th itme.
    
    Parameters
    ----------
    src : str
        The remote *source* for the file, any valid URL should work.
    dest : str
        The *destination* location to save the downloaded file.
        
    Returns
    -------
    str
        A string representing the local location of the file.
    """
    
    url = urlparse(src) # We assume that this is some kind of valid URL 
    fn  = os.path.split(url.path)[-1] # Extract the filename
    dfn = os.path.join(dest,fn) # Destination filename
    
    # Check if dest+filename does *not* exist -- 
    # that would mean we have to download it!
    if not os.path.isfile(dfn):
        
        print(f"{dfn} not found, downloading!")

        # Convert the path back into a list (without)
        # the filename -- we need to check that directories
        # exist first.
        path = os.path.split(dest)
        
        # Create any missing directories in dest(ination) path
        # -- os.path.join is the reverse of split (as you saw above)
        # but it doesn't work with lists... so I had to google how
        # to use the 'splat' operator! os.makedirs creates missing
        # directories in a path automatically.
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            
        # Download and write the file
        with open(dfn, "wb") as file:
            response = get(src)
            file.write(response.content)
            
        print('Done downloading...')

    else:
        print(f"Found {dfn} locally!")

    return dfn

In [ ]:
ddir  = os.path.join('data','geo') # destination directory
spath = 'https://github.com/jreades/i2p/blob/master/data/src/' # source path

boros = gpd.read_file( cache_data(spath+'Boroughs.gpkg?raw=true', ddir) )
print('Done.')

In [ ]:
#checking projection
print(boros.crs)

In [ ]:
boros.plot()

In [ ]:
boros.head()

In [ ]:
#creating the geodataframe
geotypes=typecount.set_index('ONS code').join(boros.set_index('GSS_CODE'))
gdf = gpd.GeoDataFrame(geotypes, crs = "EPSG:27700")

### Mapping % of Entire Homes listings in May 2019

In [ ]:
fig, ax = plt.subplots(1,1)

gdf.plot(column = 'Percent_Entire_Home',
        ax=ax, 
        legend=True)
        #legend = {'label':"% of London's Entire Homes Listings", 'orientation':"horizontal"})

In [ ]:
fig, ax = plt.subplots(1,1)

gdf.plot(column = 'Density_of_Entire_Homes',
        ax=ax, 
        legend=True)
        #legend = {'label':"% of London's Entire Homes Listings", 'orientation':"horizontal"})

Evidentally, the majority of entire homes listings are in Westminster

In [ ]:
#creating geodataframe of listings

entirehomesgdf = gpd.GeoDataFrame(entirehomeslatlong, 
      geometry=gpd.points_from_xy(entirehomeslatlong['longitude'], entirehomeslatlong['latitude'], crs='epsg:4326'))
entirehomesgdf = entirehomesgdf.to_crs('epsg:27700')
entirehomesgdf.plot(column='room_type', cmap='plasma', markersize=1, figsize=(8,6));

## Mapping Empty Homes in London

In [ ]:
emptyhomes.head()

In [ ]:
emptyhomes['Density of Empty Homes'] = emptyhomes['2018']/totalhomes['2018']
emptyhomes.columns = emptyhomes.columns.map(lambda x: str(x) + '_empty')
emptyhomes.head()

In [ ]:
geotypes.head()

In [ ]:
#joining empty homes to the geodataframe
geotypes=geotypes.set_index('LAD_Name').join(emptyhomes.set_index('Area_empty'))
gdf = gpd.GeoDataFrame(geotypes, crs = "EPSG:27700")

In [ ]:
fig, ax = plt.subplots(1,1)

gdf.plot(column = 'Density of Empty Homes_empty',
        ax=ax, 
        legend=True)
        #legend = {'label':"% of London's Entire Homes Listings", 'orientation':"horizontal"})

## Mapping Empty Homes and Airbnb Entire Homes Listings

In [ ]:
geotypes.info()

In [ ]:
geotypes['Total empty and entire homes 2018'] = geotypes['Entire_Home']+geotypes['2018_empty']
geotypes['Percent Unavailable 2018'] = geotypes['Total empty and entire homes 2018'] / geotypes['2018']
geotypes['Total empty and entire homes 2019'] = geotypes['Entire_Home']+geotypes['2019_empty']
geotypes['Percent Unavailable 2019'] = geotypes['Total empty and entire homes 2019'] / geotypes['2018']
geotypes.info()

In [ ]:
gdf = gpd.GeoDataFrame(geotypes, crs = "EPSG:27700")

fig, ax = plt.subplots(1,1)

gdf.plot(column = 'Percent Unavailable 2018',
        ax=ax, 
        legend=True)
        #legend = {'label':"% of London's Entire Homes Listings", 'orientation':"horizontal"})

In [ ]:
gdf = gpd.GeoDataFrame(geotypes, crs = "EPSG:27700")

fig, ax = plt.subplots(1,1)

gdf.plot(column = 'Percent Unavailable 2019',
        ax=ax, 
        legend=True)
        #legend = {'label':"% of London's Entire Homes Listings", 'orientation':"horizontal"})

## Compare the above to the housing shortage

In [ ]:
#Plot graphs of empty homes year on year 

emptyhomes.loc["Total"] = emptyhomes.sum(numeric_only=True)
emptyhomes


## Relationship Between Empty Homes and Entire Homes Listings

# 

In [ ]:
# Scatterplot of number of empty homes and number of entire homes listings by borough

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) 
ax.set_facecolor('xkcd:white')

x = geotypes['2019_empty']
y = geotypes['Entire_Home']
plt.scatter(x, y)

z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

plt.show()

#add the rsq and equation of the line

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) 
ax.set_facecolor('xkcd:white')

plt.plot(x,y,"o", ms=5, mec="k", color = 'darkblue')
z = np.polyfit(x, y, 1)
y_hat = np.poly1d(z)(x)

plt.plot(x, y_hat, "r--", lw=2, color='lightblue')
text = f"$y={z[0]:0.2f}\;x{z[1]:+0.2f}$\n$R^2 = {r2_score(y,y_hat):0.2f}$"
plt.gca().text(0.05, 0.95, text,transform=plt.gca().transAxes,
     fontsize=14, verticalalignment='top')
ax.set_ylabel('Level4 Qualifications (Female)')
ax.set_xlabel('Level4 Qualifications (Male)')